In [1]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import tarfile
data_dir='/content/drive/MyDrive/ColabNotebooks/refcocog.tar.gz'
# Extract data
tar = tarfile.open(data_dir)
tar.extractall('dataset/')


In [3]:
!pip install torch
!pip install torchvision
!pip install SummaryWriter
!pip install tensorboard
!pip install scikit-image
!pip install matplotlib
!pip install Lambda
!pip install opencv-python
!pip install torchmetrics

!python -m pip install pyyaml==5.1
import sys, os, distutils.core



import torch
torch.cuda.current_device()
import torchvision
import torch.nn.functional as F
import torchvision.transforms as T
from torchvision import ops
from torch.utils.data import Dataset
from torch.utils.tensorboard import SummaryWriter
from posixpath import split
import json
import tarfile
import io
import pickle
from PIL import Image

import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import jaccard_score



# import some common libraries
import cv2
#from google.colab.patches import cv2_imshow

device = "cuda" if torch.cuda.is_available() else "cpu"
temperature = 1.0
torch.autograd.set_detect_anomaly(True)
data = {}

ERROR: Could not find a version that satisfies the requirement SummaryWriter (from versions: none)
ERROR: No matching distribution found for SummaryWriter
  Using cached PyYAML-5.1.tar.gz (274 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [4]:
!pip install git+https://github.com/openai/CLIP.git
from clip import clip

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-d8bq1xnh
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-d8bq1xnh
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Preparing metadata (setup.py) ... done


In [5]:
def get_datasets(refs, train = True):
  ref_data = []
  for val in refs:
    if train:
      if val['split'] == 'train' or val['split'] == 'val':
        ref_data.append(val)
    else:
      if val['split'] == 'test':
        ref_data.append(val)

  '''for i,v in enumerate(datasets):
    training_data, test_data = v'''

  return ref_data


def preProcess_datasets(data_dir):
  train_dataset = {}
  test_dataset = {}
  f = open(f'{data_dir}/annotations/refs(umd).p', 'rb')
  data['refs'] = pickle.load(f)
  '''if train:
    refs = get_datasets(data['refs'], train=True)
  else:
    refs = get_datasets(data['refs'], train=False)'''
  instances_file = os.path.join(f'{data_dir}/annotations/instances.json')
  instances = json.load(open(instances_file, 'r'))
  data['images'] = instances['images']
  data['annotations'] = instances['annotations']
  #print(data['refs'][0])
  #print(data['images'][0])
  #print(data['annotations'][0])
  c=0
  d=0
  for key,val in enumerate(data['refs']):
    for v in data['images']:
      if val['image_id'] == v['id']:
        fname = os.path.join(data_dir+'/images/', v['file_name'])
        if os.path.exists(fname):
          if val['split'] == 'train' or val['split'] == 'val':
            train_dataset[c] = {}
            train_dataset[c]['image'] = v['file_name']
            train_dataset[c]['captions'] = [t['raw'] for t in val['sentences']]
            train_dataset[c]['id'] = val['image_id']
            break
          else:
            test_dataset[d] = {}
            test_dataset[d]['image'] = v['file_name']
            test_dataset[d]['captions'] = [t['raw'] for t in val['sentences']]
            test_dataset[d]['id'] = val['image_id']
            break
    for ann in data['annotations']:
      if val['ann_id'] == ann['id']:
        if val['split'] == 'train' or val['split'] == 'val':
          train_dataset[c]['bbox'] = ann['bbox']
          c+=1
          break
        else:
          test_dataset[d]['bbox'] = ann['bbox']
          d+=1
          break
    if c>=5000 and d>=5000:
      break

  return train_dataset, test_dataset


def resize_bounding_box(bbox, original_size, new_size):
  x_min, y_min, x_max, y_max = bbox
  orig_width, orig_height = original_size
  new_width, new_height = new_size

  # Calculate the scaling factors
  width_ratio = new_width / orig_width
  height_ratio = new_height / orig_height

  # Resize the bounding box coordinates
  new_x_min = int(x_min * width_ratio)
  new_y_min = int(y_min * height_ratio)
  new_x_max = int(x_max * width_ratio)
  new_y_max = int(y_max * height_ratio)

  return new_x_min, new_y_min, new_x_max, new_y_max

def xywh_to_xyxy(boxes):
  xmin = boxes[0]
  ymin = boxes[1]
  w = boxes[2]
  h = boxes[3]
  xmax = xmin + w
  ymax = ymin + h
  return [xmin, ymin, xmax, ymax]



def visualise_result(img, txt, bbox):
  resize_image = T.Resize(100)
  img = resize_image(img)
  convert_tensor = T.ToTensor()
  image = convert_tensor(img).to(device)
  image = image.clone().detach()
  image = image.type(torch.ByteTensor).to(device)
  boxs = torch.tensor(bbox,dtype=torch.int).to(device)
  boxes = boxs.reshape([1,4])
  boxes = torchvision.ops.box_convert(boxes, "cxcywh", "xyxy").to(device)
  image = draw_bounding_boxes(image=image, boxes=boxes, width=2, colors=(0,0,255), fill=True).to(device)
  image = image.permute(1,2,0)

In [6]:
class RefCOCOgDataset(Dataset):
    def __init__(self, dataset, transform, target_transform, data_dir='dataset/refcocog'):
        super(RefCOCOgDataset, self).__init__()
        self.data_dir = data_dir
        self.transform = transform
        self.target_transform = target_transform
        self.dataset = dataset



    def __getitem__(self, idx):
        data_item = {}
        fname = os.path.join(self.data_dir+'/images/', self.dataset[idx]['image'])
        image = Image.open(fname).convert('RGB')
        shape_0 = list(image.size)
        if self.transform:
            data_item['image'] = self.transform(image)
        if self.target_transform:
            data_item['captions'] = clip.tokenize([sent for desc in self.dataset[idx]['captions'] for sent in desc])
            bb = xywh_to_xyxy(self.dataset[idx]['bbox'])
            bbox = resize_bounding_box(bb, shape_0, [224,224])
            data_item['bbox'] = torch.tensor(bbox)
        return data_item


    def __len__(self):
        return len(self.dataset)



In [7]:
def get_cost_function():
  #cost_function = torch.nn.SmoothL1Loss()
  cost_function = torch.nn.SmoothL1Loss()
  return cost_function

def get_iou_torch(ground_truth, pred):
  # Coordinates of the area of intersection.

  ix1 = torch.max(ground_truth[0][0], pred[0][0])
  iy1 = torch.max(ground_truth[0][1], pred[0][1])
  ix2 = torch.min(ground_truth[0][2], pred[0][2])
  iy2 = torch.min(ground_truth[0][3], pred[0][3])

  # Intersection height and width.
  i_height = torch.max(iy2 - iy1 + 1, torch.tensor(0.))
  i_width = torch.max(ix2 - ix1 + 1, torch.tensor(0.))

  area_of_intersection = i_height * i_width

  # Ground Truth dimensions.
  gt_height = ground_truth[0][3] - ground_truth[0][1] + 1
  gt_width = ground_truth[0][2] - ground_truth[0][0] + 1

  # Prediction dimensions.
  pd_height = pred[0][3] - pred[0][1] + 1
  pd_width = pred[0][2] - pred[0][0] + 1

  area_of_union = gt_height * gt_width + pd_height * pd_width - area_of_intersection

  iou = (area_of_intersection + 1e-5) / (area_of_union + 1e-5) # Adding a small epsilon to avoid division by zero
  #average_iou = torch.mean(iou)

  return iou.detach().cpu().numpy()



In [8]:
def get_optimizer(model, lr, wd, momentum):
  optimizer = torch.optim.SGD([
      {'params': model.parameters(), 'lr': lr}
  ], lr=lr, weight_decay=wd, momentum=momentum)

  #optimizer = torch.optim.Adam(model.parameters(), lr=lr)

  return optimizer

In [9]:
def training_step(net, data_loader, optimizer, cost_function, device='cuda'):
    samples = 0.0
    cumulative_loss = 0.0
    cumulative_accuracy = 0.0
    batch_ious = []
    # set the network to training mode
    net = net.float()
    net.train()

  # iterate over the training set
    for batch_idx, batch in enumerate(data_loader):

      #load data into GPU
      batch['image'] = batch['image'].to(device)
      batch['captions'] = batch['captions'].to(device)
      target_bbox = batch['bbox'].to(device)
      # forward pass
      out_bbox = net(batch['image'], batch['captions'])

      # loss computation
      loss = cost_function(out_bbox, target_bbox)
      # fetch prediction and loss value
      # backward pass
      loss.requires_grad = True
      loss.backward()

      # parameters update
      optimizer.step()

      # gradients reset
      optimizer.zero_grad()

      # fetch prediction and loss value
      samples += batch['bbox'].shape[0]
      cumulative_loss += loss.item()



      #iou_accuracy = get_iou_torch(target_bbox, out_bbox)
      iou_accuracy = torchvision.ops.box_iou(target_bbox, out_bbox).detach().cpu().numpy().sum().item()
      #print(iou.detach().cpu().numpy())
      #print(print(iou.detach().cpu().numpy().sum().item()))
      #iou_accuracy = iou_pytorch(out_bbox, target_bbox)
      #batch_ious.append(iou_accuracy.sum().item())
      batch_ious.append(iou_accuracy)
      #cumulative_accuracy += iou_accuracy

      # Calculate the average IoU across all batches
    if batch_ious:
        # Calculate the average IoU across all batches
        cumulative_accuracy = np.mean(batch_ious)
    else:
        cumulative_accuracy = 0.0 # Set to zero if there are no elements


    return cumulative_loss/samples, cumulative_accuracy

def test_step(net, data_loader, cost_function, device='cuda'):
    samples = 0.0
    cumulative_loss = 0.0
    cumulative_accuracy = 0.0
    batch_ious = []

    net = net.float()
    # set the network to evaluation mode
    net.eval()

    # disable gradient computation (we are only testing, we do not want our model to be modified in this step!)
    with torch.no_grad():
        # iterate over the test set

      for batch_idx, batch in enumerate(data_loader):

        #load data into GPU
        batch['image'] = batch['image'].to(device)
        batch['captions'] = batch['captions'].to(device)
        target_bbox = batch['bbox'].to(device)

        # forward pass
        out_bbox = net(batch['image'], batch['captions'])

        # loss computation
        loss = cost_function(out_bbox, target_bbox)
        # fetch prediction and loss value

        samples += batch['bbox'].shape[0]
        cumulative_loss += loss.item() # Note: the .item() is needed to extract scalars from tensors


        #iou_accuracy = get_iou_torch(target_bbox, out_bbox)
        iou_accuracy = torchvision.ops.box_iou(target_bbox, out_bbox).detach().cpu().numpy().sum().item()
        #print(iou.detach().cpu().numpy())
        #print(print(iou.detach().cpu().numpy().sum().item()))
        #iou_accuracy = iou_pytorch(out_bbox, target_bbox)
        #batch_ious.append(iou_accuracy.sum().item())
        batch_ious.append(iou_accuracy)
        #cumulative_accuracy += iou_accuracy


    if batch_ious:
        # Calculate the average IoU across all batches
        cumulative_accuracy = np.mean(batch_ious)
    else:
        cumulative_accuracy = 0.0  # Set to zero if there are no elements

    return cumulative_loss/samples, cumulative_accuracy

In [10]:
def _make_grid_(input):
    # Get the width and height of the output feature map
    _, height, width = input.size()

    # Determine the size of each grid cell
    grid_size = height // 4
    grid_cells = []
    boxes = torch.zeros(4, 4, 4)
    for i in range(4):
        for j in range(4):
            x1 = i * grid_size
            y1 = j * grid_size
            x2 = x1 + grid_size
            y2 = y1 + grid_size
            boxes[i, j, :] = torch.tensor([float(x1), float(y1), float(x2), float(y2)])
            grid_cell = input[:, x1:x2, y1:y2]
            grid_cells.append(grid_cell)
    grid_cells = torch.stack(grid_cells)
    return grid_cells, boxes


def _make_grid(input):
    # Get the width and height of the output feature map
    _, width, height = input.size()
    # Determine the size of each grid cell
    grid_size = height // 4
    grid_cells = []
    grid_coordinates = []
    for i in range(4):
        for j in range(4):
            # Calculate the coordinates for the current grid cell
            x1 = i * grid_size
            y1 = j * grid_size
            x2 = x1 + grid_size
            y2 = y1 + grid_size
            # Extract the region corresponding to the grid cell
            grid_cell = input[:, x1:x2, y1:y2]
            grid_coordinates.append(torch.tensor([float(x1),float(y1),float(x2),float(y2)]))
            grid_cells.append(grid_cell)
    grid_cells = torch.stack(grid_cells)
    grid_coordinates = torch.stack(grid_coordinates)
    return grid_cells, grid_coordinates


class Grid(torch.nn.Module):
    def __init__(self):
        super(Grid, self).__init__()

    def forward(self, x):
        res = []
        for img in x:
            res.append(_make_grid(img))
        return res


class CustomCLIP(torch.nn.Module):
  def __init__(self, num_classes: int = 10, bias=False):
    super().__init__()
    in_features = 1024
    out_features = 1024
    #self.model = model.children()[:3]
    #layers = list(model.children())[:4]
    #self.features1 = torch.nn.Sequential(*layers[:2])
    #self.features2 = torch.nn.Sequential(*layers[2:])

    self.encoder = model.visual.float()
    self.conv1 = self.encoder.conv1
    self.bn1 = self.encoder.bn1
    self.relu1 = self.encoder.relu1
    self.avgpool = self.encoder.avgpool
    #self.mlp = self.encoder.mlp
    self.dropout = torch.nn.Dropout(0.1)
    self.dtype = self.encoder.type(self.conv1.weight.dtype)
    self.layer1 = self.encoder.layer1
    self.attnpool = self.encoder.attnpool


    self.text_encoder = self.encode_text

    self.grid = Grid()

    # add a bottleneck
    self.image_encoder = torch.nn.Sequential(
      self.conv1,
      self.bn1,
      self.relu1,
      torch.nn.Dropout(0.1),
      self.avgpool,
      torch.nn.Flatten(),
      torch.nn.Linear(20000, num_classes, bias=bias),
      torch.nn.Dropout(0.1),
    )

    '''self.classifier = torch.nn.Sequential(
      torch.nn.Linear(25, num_classes, bias=bias),
      torch.nn.Dropout(0.1),
      torch.nn.BatchNorm2d(32),
    )

    self.text_classifier = torch.nn.Sequential(
      torch.nn.Linear(512, num_classes, bias=bias),
      torch.nn.Dropout(0.1),
    )'''

  def encode_text(self, text):
      x = model.token_embedding(text)
      #x = x + model.positional_embedding
      x = model.ln_final(x)

      return x

  def forward(self, img, cap):
    grids = self.grid(img)
    #_, height, width = img.size()
    aspect_ratio = 224 / 224
    target_width = 100
    target_height = int(target_width / aspect_ratio)
    boxes = grids[0][1]
    boxes = boxes.to(device)
    similar = []
    for idx, (gd, _) in enumerate(grids):
      x_ = []
      for g in gd:

          x = F.interpolate(g.unsqueeze(0), size=(target_width, target_height), mode='bilinear', align_corners=False)
          with torch.no_grad():
            x = self.image_encoder(x)
          #x = self.classifier(x)
          x /= x.norm(dim=-1, keepdim=True)
          x_.append(x)

      with torch.no_grad():
        y = self.text_encoder(cap[idx].unsqueeze(0))
        #y = self.text_classifier(y)
      y /= y.norm(dim=-1, keepdim=True)


      im_ = torch.cat(x_)

      #print(im_.shape)
      #print(y.shape)
      #im_ = im_.squeeze()

      #target_len = y.size(1) - im_.size(1)
      #im_ = F.pad(im_, (0,0,0,0,0,target_len,0,0), value=0)
      #target_len = y.size(3) - im_.size(3)
      #im_ = F.pad(im_, (0,target_len,0,0,0,0,0,0), value=0)
      target_len = y.size(3) - im_.size(1)
      im_ = F.pad(im_, (0,target_len,0,0), value=0)
      #print(im_.shape)
      #print(y.shape)

      similarity = F.softmax(im_ @ y.mT, dim=0)
      #similarity = F.softmax(x_ @ y.mT, dim=-1)
      top_values, top_indices = similarity.topk(1, dim=0)
      #print(boxes[top_indices].shape)
      similar.append(boxes[top_indices])
    out_bbox = torch.cat(similar)
    out_bbox = out_bbox[:, 0, 0, 0, :]
    #print(out_bbox.shape)

    return out_bbox

In [11]:
def pad_sequence(batch):
  return torch.nn.utils.rnn.pad_sequence(batch, batch_first=True, padding_value=0)
def my_collate_fn(batch):

  return {
      'image': pad_sequence([x['image'] for x in batch]),
      'captions': pad_sequence([x['captions'] for x in batch]),
      'bbox': pad_sequence([x['bbox'] for x in batch])
      }


def get_data(data_dir, batch_size, test_batch_size, transform=True, target_transform=False):


    if transform:
        # convert the PIL images to Tensors
        transform = T.Compose([
            T.Resize((224, 224)),
            T.RandomCrop(size=224),
            T.RandomHorizontalFlip(),
            T.RandomRotation(degrees=30),
            T.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
            T.ToTensor(),
            T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])
    else:
          # prepare data transformations and then combine them sequentially
        transform = preprocess
    if target_transform:
        target_transform = T.Compose([
                                 lambda y: torch.zeros(10, dtype=torch.float).scatter_(0, torch.tensor(y), value=1)])
    else:
        target_transform = None

  # load data
    train_dataset, test_dataset = preProcess_datasets(data_dir)
    #print(len(train_dataset))
    #print(len(test_dataset))
    full_training_data = RefCOCOgDataset(dataset=train_dataset, transform=transform, target_transform=target_transform, data_dir=data_dir)

    #test_dataset = preProcess_datasets(data_dir, train=False)
    global test_data
    test_data = RefCOCOgDataset(dataset=test_dataset, transform=transform, target_transform=target_transform, data_dir=data_dir)


    #evens = list(range(0, len(full_training_data), 2))
    #training_data2 = torch.utils.data.Subset(full_training_data, evens)
    #training_data2 = torch.utils.data.Subset(full_training_data, range(5000))
    #test_data2 = torch.utils.data.Subset(test_data, range(5000))

  # create train and validation splits
    #num_samples = len(training_data2)
    num_samples = len(full_training_data)

    training_samples = int(num_samples * 0.8 + 1)
    validation_samples = num_samples - training_samples

    #training_data, validation_data = torch.utils.data.random_split(training_data2, [training_samples, validation_samples])
    training_data, validation_data = torch.utils.data.random_split(full_training_data, [training_samples, validation_samples])
    print(len(training_data))

    print(len(validation_data))
    print(len(test_data))
  # initialize dataloaders_collate
    train_loader = torch.utils.data.DataLoader(training_data, batch_size, shuffle=True, num_workers=2, collate_fn=my_collate_fn)
    val_loader = torch.utils.data.DataLoader(validation_data, test_batch_size, shuffle=False, num_workers=2, collate_fn=my_collate_fn)
    test_loader = torch.utils.data.DataLoader(test_data, test_batch_size, shuffle=False, num_workers=2, collate_fn=my_collate_fn)
    #test_loader = torch.utils.data.DataLoader(test_data2, test_batch_size, shuffle=False, num_workers=2, collate_fn=my_collate_fn)




    # pre-train model on zero-shot transfer learning
    #test_accuracy = test_step_zero_shot_clip(model, test_loader, texts_z)


    # evaluate accuracy on zero-shot learning

    #print("Test accuracy {:.2f}".format(test_accuracy))

    return train_loader, val_loader, test_loader


In [12]:
# tensorboard logging utilities
def log_values(writer, step, loss, accuracy, prefix):
  writer.add_scalar(f"{prefix}/loss", loss, step)
  writer.add_scalar(f"{prefix}/accuracy", accuracy, step)

In [13]:
model, preprocess = clip.load("RN50")
model = model.float()
modified_model = CustomCLIP(num_classes=10).to(device)

In [14]:
# main funcition
def main(
      root='/content/dataset/refcocog/',
      data_dir='dataset/refcocog',
      batch_size=64,
      test_batch_size=64,
      num_classes=10,
      learning_rate=0.01,
      weight_decay=1e-5,
      momentum=0.9,
      epochs=10
      #epochs=3
    ):
  writer = SummaryWriter(log_dir="runs/exp1")
  #global test_loader
  # train clip on zero-shot learning and instantiates dataloaders
  train_loader, val_loader, test_loader = get_data(data_dir=data_dir, batch_size=batch_size, test_batch_size=test_batch_size, transform=True, target_transform=True)


  # instantiate the network and move it to the chosen device (GPU)


  # instantiate the optimizer
  optimizer = get_optimizer(modified_model, learning_rate, weight_decay, momentum)

  # define the cost function
  cost_function = get_cost_function()

  # evaluate accuracy of modified model on zero-shot learning
  #test_accuracy = test_step_zero_shot_clip(model, test_loader, texts_z).to(device)

  #print(cosine_similarity(images_z, texts_z))
  #print("Test accuracy {:.2f}".format(test_accuracy))

  # computes evaluation results before training
  print('Before training:')
  train_loss, train_accuracy = test_step(modified_model, train_loader, cost_function)
  val_loss, val_accuracy = test_step(modified_model, val_loader, cost_function)
  test_loss, test_accuracy = test_step(modified_model, test_loader, cost_function)

  # log to TensorBoard
  log_values(writer, -1, train_loss, train_accuracy, "train")
  log_values(writer, -1, val_loss, val_accuracy, "validation")
  log_values(writer, -1, test_loss, test_accuracy, "test")

  print('\tTraining loss {:.5f}, Training accuracy {:.2f}'.format(train_loss, train_accuracy))
  print('\tValidation loss {:.5f}, Validation accuracy {:.2f}'.format(val_loss, val_accuracy))
  print('\tTest loss {:.5f}, Test accuracy {:.2f}'.format(test_loss, test_accuracy))
  print('-----------------------------------------------------')

  # for each epoch, train the network and then compute evaluation results
  for e in range(epochs):

    train_loss, train_accuracy = training_step(modified_model, train_loader, optimizer, cost_function)
    val_loss, val_accuracy = test_step(modified_model, val_loader, cost_function)

    # logs to TensorBoard
    log_values(writer, e, val_loss, val_accuracy, "Validation")

    print('Epoch: {:d}'.format(e+1))
    print('\tTraining loss {:.5f}, Training accuracy {:.2f}'.format(train_loss, train_accuracy))
    print('\tValidation loss {:.5f}, Validation accuracy {:.2f}'.format(val_loss, val_accuracy))
    print('-----------------------------------------------------')

    # compute final evaluation results
    print('After training:')
    train_loss, train_accuracy = test_step(modified_model, train_loader, cost_function)
    val_loss, val_accuracy = test_step(modified_model, val_loader, cost_function)
    test_loss, test_accuracy = test_step(modified_model, test_loader, cost_function)

    # log to TensorBoard
    log_values(writer, epochs, train_loss, train_accuracy, "train")
    log_values(writer, epochs, val_loss, val_accuracy, "validation")
    log_values(writer, epochs, test_loss, test_accuracy, "test")

    print('\tTraining loss {:.5f}, Training accuracy {:.2f}'.format(train_loss, train_accuracy))
    print('\tValidation loss {:.5f}, Validation accuracy {:.2f}'.format(val_loss, val_accuracy))
    print('\tTest loss {:.5f}, Test accuracy {:.2f}'.format(test_loss, test_accuracy))
    print('-----------------------------------------------------')


  # closes the logger
  writer.close()
  torch.save(modified_model.state_dict(), 'model.pth')


In [15]:
torch.cuda.empty_cache()
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:<1024>"
os.environ["CUDA_LAUNCH_BLOCKING"]="1"

In [16]:
!pip install scikit-learn
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import f1_score

def calculate_accuracy(output, target):


  f1_scr = f1_score(target, output, average='weighted')

  cm = confusion_matrix(target, output)
  TP = cm[0][0]
  FP = cm[0][1]
  FN = cm[1][0]
  TN = cm[1][1]

  #accuracy = np.sum(np.diagonal(cm)) / np.sum(cm)
  #accuracy = (TP+TN)/(TP+FP+FN+TN)
  return f1_scr

In [17]:
#torch.cuda.empty_cache()
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:<1024>"
os.environ["CUDA_LAUNCH_BLOCKING"]="1"
os.environ["TORCH_USE_CUDA_DSA"]="1"

In [ ]:
main()

4001
999
5023
Before training:
	Training loss 1.39093, Training accuracy 94.81
	Validation loss 1.41712, Validation accuracy 87.07
	Test loss 1.39667, Test accuracy 80.69
-----------------------------------------------------
Epoch: 1
	Training loss 1.39038, Training accuracy 94.49
	Validation loss 1.41712, Validation accuracy 87.07
-----------------------------------------------------
After training:
	Training loss 1.39072, Training accuracy 94.90
	Validation loss 1.41712, Validation accuracy 87.07
	Test loss 1.39667, Test accuracy 80.69
-----------------------------------------------------
Epoch: 2
	Training loss 1.39034, Training accuracy 94.99
	Validation loss 1.41712, Validation accuracy 87.07
-----------------------------------------------------
After training:
	Training loss 1.39144, Training accuracy 95.12
	Validation loss 1.41712, Validation accuracy 87.07
	Test loss 1.39667, Test accuracy 80.69
-----------------------------------------------------
Epoch: 3
	Training loss 1.390

In [ ]:
def draw_corner_rect(bb, color='red'):
    bb = np.array(bb, dtype=np.float32)
    print(bb)
    return plt.Rectangle((bb[1], bb[0]), bb[3]-bb[1], bb[2]-bb[0], color=color,
                         fill=False, lw=3)

def draw_bbx(bb, color):
    plt.gca().add_patch(draw_corner_rect(bb, color))


def inference(model, query, image, gd_bbox):

    model.load_state_dict(torch.load('model.pth'))
    model.eval()

    with torch.no_grad():

      bbox = model(image.unsqueeze(0).to(device), query.unsqueeze(0).to(device))
      print('inference')
      print(bbox)
      print(bbox.shape)

      #caption = tokenizer.decode(query[0], skip_special_tokens=False)


      image = image.permute(1,2,0)
      img = cv2.cvtColor(image.numpy(), cv2.COLOR_BGR2RGB)

      out_bb = bbox.detach().cpu().numpy()
      out_bb = out_bb.astype(int)
      print(out_bb)
      print(out_bb.shape)
      plt.imshow(img)

      draw_bbx(out_bb[0],'red')

      plt.axis('off')
      plt.show()






In [ ]:
x = test_data[4]


inference(modified_model,
             query=x['captions'],
             image=x['image'],
              gd_bbox=x['bbox'])
